# Subtitle converter

[![Open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/naipofo/VobSubConvertJp/blob/main/VobSubConvertJp.ipynb)

This notebook will convert your VobSub subtitles (.idx + .sub) to conventional text subtitles(SubStation Alpha).

If you have subtitles in .sup format, You can convert them to a .idx + .sub combo with [bdsup2subpp](https://github.com/mjuhasz/BDSup2Sub).

If you are using google colab, remember to enable GPU acceleration to speed up the convertion.

Check out my [github repo](https://github.com/naipofo/VobSubConvertJp) for more information

Run the cells below to continiue ⬇️

In [ ]:
#@title **Dependeces**
# @markdown Run this cell to install all dependeces. It might take some time.<br>
# @markdown You only need to run this cell once.
import os.path
if (not os.path.isfile("~/.cargo/bin/cargo")):
  !curl https://sh.rustup.rs -sSf | sh -s -- -y
!~/.cargo/bin/cargo install vobsub2png
!pip install manga-ocr
!pip install tqdm
!pip install pysubs2
from manga_ocr import MangaOcr
mocr = MangaOcr()



In [ ]:
#@title **Setup**
# @markdown Use the file browser on the left to upload your subtitle files.<br>
# @markdown Make sure you have a matching .sub file in the same directory!<br>
subtitle_name = "hanabi.idx" # @param {type:"string"}
!rm -rf ./subimg
!/root/.cargo/bin/vobsub2png -o subimg {subtitle_name}

In [ ]:
#@title **Convert**
# @markdown <font size=5>Run this cell to convert the subtitle file. It will be automatically
# @markdown downloaded after the conversion finishes.

from tqdm import tqdm
import pysubs2
from pysubs2 import Alignment, Color, SSAFile, SSAStyle, SSAEvent, make_time
import json

with open('subimg/index.json') as user_file:
  subs_meta = json.load(user_file)

subs = SSAFile()

for m in tqdm(subs_meta['subtitles']):
  subs.append(SSAEvent(
      start=make_time(s=m["start"]),
      end=make_time(s=m["end"]),
      text=mocr("subimg/"+m["path"])))

subs.info = {
    "; Converted by https://github.com/naipofo/VobSubConvertJp ":"",
    **subs.info
}

file_name = f"{subtitle_name[:-4]}.ass"
import os.path
if (os.path.isfile(file_name)):
  !rm {file_name}
subs.save(file_name, 'utf-8', 'ass')
print(f"generated {file_name}")
from google.colab import files
files.download(file_name)